In [95]:
import numpy as np
from tensorpack import tfutils
import tensorflow as tf
from scipy import spatial

In [4]:
MFCC_SIZE = 20
N_INP_FRMS = 50

utt_data = np.load('exp/test_utt.npy')
n_frms = utt_data.shape[0] - N_INP_FRMS + 1
batch_x = np.zeros((n_frms, N_INP_FRMS*MFCC_SIZE))
for i in range(n_frms):
    batch_x[i] = utt_data[i:(i+N_INP_FRMS), 0:MFCC_SIZE].reshape(1, N_INP_FRMS*MFCC_SIZE)

In [80]:
checkpoint_path = tf.train.latest_checkpoint("./model_data/")
saver = tf.train.import_meta_graph(checkpoint_path + ".meta", import_scope=None)
print("Running network inference...")
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    nn_dvector = sess.run("dense_1/BiasAdd:0", feed_dict={"maxout_dense_1_input:0": batch_x})
    bn1 = sess.run("batch_normalization_1/batchnorm_1/add_1:0", feed_dict={"maxout_dense_1_input:0": batch_x})
    bn2 = sess.run("batch_normalization_2/batchnorm_1/add_1:0", feed_dict={"maxout_dense_1_input:0": batch_x})
    bn3 = sess.run("batch_normalization_3/batchnorm_1/add_1:0", feed_dict={"maxout_dense_1_input:0": batch_x})
    bn4 = sess.run("batch_normalization_4/batchnorm_1/add_1:0", feed_dict={"maxout_dense_1_input:0": batch_x})
    bn5 = sess.run("batch_normalization_5/batchnorm_1/add_1:0", feed_dict={"maxout_dense_1_input:0": batch_x})
bns = [bn1,bn2,bn3,bn4,bn5]

Running network inference...
INFO:tensorflow:Restoring parameters from ./model_data/model-1


In [102]:
vrs = tfutils.varmanip.load_chkpt_vars('model_data/checkpoint')
np.save('model_data/model-1.npy', vrs)

In [93]:
def bn(inp, var_ema, mean_ema, gamma, beta):
    scale = gamma/np.sqrt(var_ema+1e-5)
    scaled_inp = inp*scale
    scaled_mean_ema = scale*mean_ema
    bias = beta-scaled_mean_ema
    out = scaled_inp + bias
    return out

def maxout(W, b, inp):
    t1 = np.dot(inp, W[0]) + b[0]
    t2 = np.dot(inp, W[1]) + b[1]
    t3 = np.dot(inp, W[2]) + b[2]
    t4 = np.dot(inp, W[3]) + b[3]
    tmax = np.maximum(np.maximum(np.maximum(t1,t2), t3), t4)
    return tmax

def fc(inp,W,b):
    return np.dot(inp,W) + b

In [115]:
vrs = np.load('model_data/model-1.npy').item()

In [116]:
inp = batch_x
for i in range(1,6):
    print("Trying maxout layer:", i)
    W = vrs['maxout_dense_%s/W' % (i,)]
    b = vrs['maxout_dense_%s/b' % (i,)]

    mx = maxout(W, b, inp)

    var_ema = vrs['batch_normalization_%s/moving_variance' % (i,)]
    mean_ema = vrs['batch_normalization_%s/moving_mean' % (i,)]
    gamma = vrs['batch_normalization_%s/gamma' % (i,)]
    beta = vrs['batch_normalization_%s/beta' % (i,)]

    inp = bn(mx, var_ema, mean_ema, gamma, beta)
    assert np.allclose(inp, bns[i-1], atol=1e-5)

W = vrs['dense_1/kernel']
b = vrs['dense_1/bias']
out = fc(inp, W, b)
assert np.allclose(out, nn_dvector, atol=1e-5)

Trying maxout layer: 1
Trying maxout layer: 2
Trying maxout layer: 3
Trying maxout layer: 4
Trying maxout layer: 5


In [100]:
g_mean = np.average(nn_dvector, axis=0)
sum_dists = 0
for i in range(10):
    m = np.load('model_data/skanda/' + str(i) + '.npy')
    m_mean = np.average(m, axis=0)
    dist = 1 - np.dot(m_mean, g_mean)/(np.linalg.norm(m_mean)*np.linalg.norm(g_mean))
    sum_dists += dist
    print(dist)
avg_dist = sum_dists/10
print('avg dist', avg_dist)

0.08466631174087524
0.07206255197525024
0.08271986246109009
0.09243404865264893
0.1268584132194519
0.07794433832168579
0.13270705938339233
0.1324428915977478
0.10052770376205444
0.09876036643981934
avg dist 0.10011235475540162
